In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.initializers import Constant 

from nltk.tokenize import TreebankWordTokenizer

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [4]:
import os
!echo '{"username":"imrankhan1386","key":"8708dd3c36c4db8c0dc44b3191edfe49"}' > ~/.kaggle/kaggle.json
!kaggle datasets download -d iarunava/imdb-movie-reviews-dataset # api copied from kaggle

100% 224M/224M [00:02<00:00, 111MB/s] 



In [5]:
!unzip /content/imdb-movie-reviews-dataset.zip

Output hidden; open in https://colab.research.google.com to view.

In [0]:
import glob
import os

from random import shuffle

def pre_process_data(filepath):
  positive_path = os.path.join(filepath, 'pos')
  negative_path = os.path.join(filepath, 'neg')
  pos_label = 1
  neg_label = 0
  dataset = []
  
  for filename in glob.glob(os.path.join(positive_path, '*.txt')):
    with open(filename, 'r') as f:
      dataset.append((pos_label, f.read()))

  for filename in glob.glob(os.path.join(negative_path, '*.txt')):
    with open(filename, 'r') as f:
      dataset.append((neg_label, f.read()))

  shuffle(dataset)
  return dataset

dataset = pre_process_data('/content/aclimdb/aclImdb/train') 

In [8]:
dataset[0]

(0,
 'I found "The Arab Conspiracy" in a bargain bin and thought I\'d uncovered a lost treasure. Folks, there\'s a reason why you don\'t hear much about this film. The plot is muddy, the pacing is slow, Cornelia Sharpe is about as vivacious as plain, cold tofu, and the ending leaves you flat. Not even Sean Connery can save this one.')

In [0]:
def tokenize_data(dataset):
  tokenizer = TreebankWordTokenizer()
  tokenized_data = []
  for val in dataset:
    tokens = tokenizer.tokenize(val[1])
    tokenized_data.append(tokens)
  return tokenized_data

tokenized_data = tokenize_data(dataset)

In [0]:
import gensim
# sentences takes tokenized data for each sentence
model = gensim.models.Word2Vec(sentences = tokenized_data, size = embedding_dims, workers = 4, min_count=1)

In [0]:
# Save the model
filename = '/content/drive/My Drive/Saved Models/imdb_embeddings_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [0]:
import os
embeddings_index = {}
f = open(os.path.join('/content/drive/My Drive/Saved Models/', 'imdb_embeddings_word2vec.txt'), encoding = 'utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coef = np.asarray(values[1:])
  embeddings_index[word] = coef
f.close()

In [0]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(sample[1] for sample in dataset)
sequences = tokenizer_obj.texts_to_sequences(sample[1] for sample in dataset)
sentiment = [sample[0] for sample in dataset]

In [0]:
maxlen = 400
batch_size = 32
embedding_dims = 300
filters = 250
epochs = 2

In [0]:
# Pad sequences
word_index = tokenizer_obj.word_index
review_pad = pad_sequences(sequences , maxlen = maxlen)

In [0]:
num_words = len(word_index)+1 #Vocabulary
embedding_matrix = np.zeros((num_words, embedding_dims))

for word,i in word_index.items():
  if i > num_words:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [13]:
print('Length of word_index : {}'.format(len(word_index)))
print('Shape of review_pad : {}'.format(review_pad.shape))
print('Length of ground Truth: {}'.format(len(sentiment)))
print('Shape of embedding_matrix : {}'.format(embedding_matrix.shape))

Length of word_index : 88582
Shape of review_pad : (25000, 400)
Length of ground Truth: 25000
Shape of embedding_matrix : (88583, 300)


In [0]:
# Train Test split
split_point = int(len(review_pad)*.8)

X_train = review_pad[:split_point]
y_train = sentiment[:split_point]

X_test = review_pad[split_point:]
y_test = sentiment[split_point:]

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN
num_neurons = 50
model = Sequential()
embedding_layer = Embedding(num_words,
                           embedding_dims, 
                           embeddings_initializer = Constant(embedding_matrix),
                           input_length = 400,
                           trainable = False)
model.add(embedding_layer)
model.add(SimpleRNN(num_neurons, return_sequences=True, input_shape = (maxlen, embedding_dims)))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 300)          26574900  
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 400, 50)           17550     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 20001     
Total params: 26,612,451
Trainable params: 37,551
Non-trainable params: 26,574,900
_________________________________________________________________


In [18]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=3, validation_data=(X_test, y_test))

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 223s 11ms/step - loss: 0.6647 - acc: 0.6865 - val_loss: 0.5479 - val_acc: 0.7654
Epoch 2/3
20000/20000 [==============================] - 212s 11ms/step - loss: 0.4864 - acc: 0.7984 - val_loss: 0.5891 - val_acc: 0.7660
Epoch 3/3
20000/20000 [==============================] - 215s 11ms/step - loss: 0.4044 - acc: 0.8342 - val_loss: 0.6528 - val_acc: 0.7704


In [0]:
model_structure = model.to_json()
with open("/content/drive/My Drive/Saved Models/NLPUsingRNN_model.json", "w") as json_file:
  json_file.write(model_structure)
model.save_weights("/content/drive/My Drive/Saved Models/NLPUsingRNN_weights.h5")

In [0]:
from keras.models import model_from_json
with open("/content/drive/My Drive/Saved Models/NLPUsingRNN_model.json", "r") as json_file:
  json_string = json_file.read()
model = model_from_json(json_string)
model.load_weights('/content/drive/My Drive/Saved Models/NLPUsingRNN_weights.h5')

In [0]:
sample_1 = ["I hate that the dismal weather had me down for so long, when will it break! Ugh, when does happiness return? The sun is blinding and the puffy clouds are too thin. I can't wait for the weekend."]

In [0]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(sample_1)
sequences = tokenizer_obj.texts_to_sequences(sample_1)

In [0]:
# Pad sequences
word_index = tokenizer_obj.word_index
review_pad = pad_sequences(sequences , maxlen = maxlen)

In [24]:
model.predict_classes(review_pad)

array([[0]], dtype=int32)